In [70]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/laknath123/PDS_final_project_animalshelter/main/data/merged_df.csv")

In [3]:
df = df[(df['indicator_col']=='both')] # get a df of values that contain an outcome so we have a label

In [4]:
df_unscaled = df[['Animal ID','Intake Type','Sex upon Intake',
     'Intake Condition','Breed_intake','age','Outcome Type']]

In [5]:
# Creating a binary outcome label for the outcome type columns
def label_creation(x):
    if x=='Adoption':
        return(1)
    else:
        return(0)

In [6]:
df_unscaled['Outcome Type']= df_unscaled['Outcome Type'].apply(label_creation)

<ipython-input-6-e9cc343bd5c6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unscaled['Outcome Type']= df_unscaled['Outcome Type'].apply(label_creation)


In [7]:
intake_cat= df_unscaled[['Intake Type']]
sex_cat = df_unscaled[['Sex upon Intake']]
condition_cat = df_unscaled[['Intake Condition']]
breed_cat = df_unscaled[['Breed_intake']]
age_num = df_unscaled[['age']]

In [67]:
df_unscaled['Intake Condition'].unique()

array(['Normal', 'Injured', 'Aged', 'Sick', 'Other', 'Medical', 'Feral',
       'Pregnant', 'Nursing', 'Behavior', 'Neonatal', 'Space', 'Panleuk',
       'Med Urgent', 'Med Attn'], dtype=object)

In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
encoder = OneHotEncoder()
scaler = StandardScaler()

intake_1hot= encoder.fit_transform(intake_cat)
sex_1hot = encoder.fit_transform(sex_cat)
cond_1hot = encoder.fit_transform(condition_cat)
breed_1hot = encoder.fit_transform(breed_cat)
age_std = scaler.fit_transform(age_num)

In [9]:
from scipy.sparse import hstack
#data = hstack((intake_1hot,sex_1hot,cond_1hot,age_std))
data = hstack((intake_1hot,sex_1hot,age_std))

In [10]:
#df_unscaled.drop(['Animal ID'],axis=1,inplace=True)

In [11]:
X,y = data.toarray(),df_unscaled['Outcome Type']

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

We arrive at a value of 18 for max depth using trial and error. For this particular value the accuracy on the validation set is the highest i.e 74.7%

In [58]:
clf_en = DecisionTreeClassifier(random_state=0,criterion="entropy",max_depth=18)
clf_en.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=18, random_state=0)

In [59]:
y_predict = clf_en.predict(X_test)

In [60]:
clf_en.get_depth()

18

In [61]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
accuracy_score(y_test,y_predict)

0.7471370887667969

The model is performing well on the validation set and correctly predicts 74.7% of the data. 

In [68]:
conf_matrix = confusion_matrix(y_true=y_test, y_pred=y_predict)
conf_matrix

array([[17369,  7286],
       [ 4174, 16492]], dtype=int64)

In [71]:
print('Precision: %.3f' % precision_score(y_test, y_predict))
print('Recall: %.3f' % recall_score(y_test, y_predict))

Precision: 0.694
Recall: 0.798


We obtained an accuracy of 74.17% on our testing data which means that almost 75% of the testing samples were correctly classified
We also got a Recall value of almost 80% which means 80% of true positives existed out of all the actual positives

In [62]:
target = list(df_unscaled['Outcome Type'].unique())
feature_names = list(df_unscaled.drop(['Outcome Type','Animal ID'],axis=1).columns)

dot_data = tree.export_graphviz(clf_en,
                                out_file=None, 
                      feature_names=feature_names,  
                      class_names=target,  
                      filled=True, rounded=True,  
                      special_characters=True)  
graph = graphviz.Source(dot_data)  

graph

ValueError: Length of feature_names, 5 does not match number of features, 12